# Diamond Prediction

After the exploratory data analysis, a prediction will be made.
The folowing steps for this prediction:
1. **Preprocessing** - Handle outliers, Feature engineering.
2. **Predictions** - predicting with validation data and then with test data, metrics.
3. **Evaluation** - Baselines for comparison.

## Libraries & settings

In [1]:
#numpy
import numpy as np

from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

#model explainability
import eli5
from eli5.sklearn import PermutationImportance

#pipeline tools
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from pipelinehelper import PipelineHelper
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
#time related
from timeit import default_timer as timer
from datetime import timedelta

#timer for entire code
start = timer()

#warning hadle
import warnings
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")

#plotly
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

#settings
pd.options.display.float_format = '{:.3f}'.format

NameError: name 'pd' is not defined

## Baseline 1: Basic approach

### Decisions:
* **Preprocessing:**
    1. **Outliers:** **continuous** - fill with median.               
    2. **Feature Engineering**: only categoric labels, OrdinalEncoder.
* **Model training** - using cross validation in validation data on a simple linear regression model.
* **Model testing** - train on whole train + validation set and use test data for results.
* **Model evaluating** - record for validation and test the following metrics:
    1. MSE - [Mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error)
    2. R2 - [R Square](https://en.wikipedia.org/wiki/Coefficient_of_determination)
    3. MAE - [Mean absolut error](https://en.wikipedia.org/wiki/Mean_absolute_error)
    4. NRMSE - [Negative root mean squared error](https://en.wikipedia.org/wiki/Root-mean-square_deviation)
    

**Preprocessing**

In [ ]:
# Preprocessing for continuous data

def Outlier_Detector(X,factor):
    X = pd.DataFrame(X).copy()
    for i in range(X.shape[1]):
        x = pd.Series(X.iloc[:,i]).copy()
        q1 = x.quantile(0.25)
        q3 = x.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - (factor * iqr)
        upper_bound = q3 + (factor * iqr)
        X.iloc[((X.iloc[:,i] < lower_bound) | (X.iloc[:,i] > upper_bound)),i] = np.nan 
    return X

#creating outlier_remover object using FunctionTransformer with factor=1.5
Outlier = FunctionTransformer(Outlier_Detector,kw_args={'factor':1.5})

#contiuous_transformer = SimpleImputer(strategy='median')

contiuous_transformer = Pipeline(steps=[
('outlier', Outlier),
('imputer', SimpleImputer(strategy='median'))
])

# building categorical transformers (worst to best)
cut_enc = OrdinalEncoder(categories=[["Fair", "Good", "Very Good", "Premium","Ideal"]])
color_enc = OrdinalEncoder(categories=[['J', 'I', 'H', 'G', 'F', 'E','D']])
clarity_enc = OrdinalEncoder(categories=[["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1",'IF']])


# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', contiuous_transformer, Continuous),
        ('cuts', cut_enc, ["cut"]),
        ('colors', color_enc, ["color"]),
        ('clarities', clarity_enc, ["clarity"])
    ])

**Model**

In [ ]:
model = LinearRegression()

**Final Pipeline**

In [ ]:
# Bundle preprocessing and modeling code in a pipeline
Baseline1 = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
Baseline1.fit(X_train2, y_train2)

**Validation Prediction**

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


# Preprocessing of validation data, get predictions
b1_val_preds = Baseline1.predict(X_val)

# Evaluate the model
b1_val_mae = mean_absolute_error(y_val, b1_val_preds)
b1_val_mse = mean_squared_error(y_val, b1_val_preds)
b1_val_r2 = r2_score(y_val, b1_val_preds)

print('MAE:', b1_val_mae)
print("MSE: ",b1_val_mse)
print("R2: ",b1_val_r2)

**Cross Validation Prediction (NRMSE)**

In [ ]:
from sklearn.model_selection import cross_val_score
CV = cross_val_score(Baseline1, X_train, y_train, cv=5, scoring = "neg_root_mean_squared_error")
print(f"validation negative root mean squared error on 5 fold cross validation: {CV}")
print(f"validation negative root mean squared error accuracy: {CV.mean()}")

**Test Prediction**

In [ ]:
Baseline1.fit(X_train, y_train)
b1_test_preds = Baseline1.predict(X_test)
b1_test_mae = mean_absolute_error(y_test, b1_test_preds)
b1_test_mse = mean_squared_error(y_test, b1_test_preds)
b1_test_r2 = r2_score(y_test, b1_test_preds)
print('MAE:', b1_test_mae)
print("MSE: ",b1_test_mse)
print("R2: ",b1_test_r2)

**Test NRMSE**

In [ ]:
print("NRMSE: ",-np.sqrt(b1_test_mse))

**Model Explainability**

The chosen method is [Permutation](https://brilliant.org/wiki/permutations/#permutations-problem-solving) Importance.
Each feature is shuffled and predicted with the rest of the normally ordered columns, where the weights are mostly changed in positive numbers means they most effect the model. negative means the model performed better with this random ordered feature, which means it's that it is useless as a predictor.

In [ ]:
x_tr = Baseline1.named_steps["preprocessor"].fit_transform(X_train)#preprocess inputs
perm = PermutationImportance(Baseline1.named_steps["model"]).fit(x_tr, y_train)#fit model
eli5.show_weights(perm, feature_names = X_train.columns.tolist())#show results

**insights:**
* depth is very important for this models prediction. 
* color has no effect on the models predictions.
* only table, z and clarity has a big effect on the model other featres less so.

### Baseline 1 Summary
The model was validated with mse, mae and r2, cross validated 5 times for NRMSE, and tested 1 time for all the metrics:
* **Mae**: validation set and test set around 1,200 and validation performed slightly better. 
* **Mse**: validation set and test set around 2,650,000 and validation performed slightly better.
* **R2**: validation set and test set around 0.83 and validation performed slightly better.
* **NRMSE**: validation set and test set around 1,600 and validation performed slightly better.

The baseline is saved for comparison as a pandas dataframe: 

In [ ]:
baseline1 = pd.DataFrame({"val_mae": b1_val_mae,"val_mse": b1_val_mse,"val_r2": b1_val_r2,"val_nrmse": CV.mean(),"test_mae": b1_test_mae,"test_mse": b1_test_mse,"test_r2": b1_test_r2, "test_nrmse": -np.sqrt(b1_test_mse)}, index=["Baseline1"])
baseline1

## Baseline 2: Preprocess Parameter tuning

### Decisions:
* **Preprocessing:**
    1. **Outliers:**  **continuous** - choose what is considered an outlier, fill with median/mean and scaling.               
    2. **Feature Engineering**: only categoric labels, OrdinalEncoder.
* **Model training** - using cross validation in validation data on a simple linear regression model.
* **Model testing** - validate with grid search and validation set then train on whole train + validation set and use test data for results.
* **Model evaluating** - record for validation and test the following metrics:
    1. MSE - [Mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error)
    2. R2 - [R Square](https://en.wikipedia.org/wiki/Coefficient_of_determination)
    3. MAE - [Mean absolut error](https://en.wikipedia.org/wiki/Mean_absolute_error)
    4. NRMSE - [Negative root mean squared error](https://en.wikipedia.org/wiki/Root-mean-square_deviation)
    

**Preprocessing**

In [ ]:
# Preprocessing for continuous data
Outlier2 = FunctionTransformer(Outlier_Detector,kw_args={'factor':np.nan})

#contiuous_transformer = SimpleImputer(strategy='median')

contiuous_transformer = Pipeline(steps=[
('outlier', Outlier2),
('imputer', SimpleImputer()),
('scaler', StandardScaler())    
])

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', contiuous_transformer, Continuous),
        ('cuts', cut_enc, ["cut"]),
        ('colors', color_enc, ["color"]),
        ('clarities', clarity_enc, ["clarity"])
    ])

**Final Pipeline**

In [ ]:
# Bundle preprocessing and modeling code in a pipeline
Baseline2 = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
Baseline2.fit(X_train2, y_train2)

In [ ]:
Baseline2.get_params().keys()

**Parameter tuning and grid search**

the factor of the outlier is being grid searched as well as impute strategy:

In [ ]:
hyperparameters = {'preprocessor__num__outlier__kw_args':[{'factor':0},{'factor':0.5},{'factor':1},{'factor':1.5},{'factor':2},{'factor':2.5},{'factor':3}],
              'preprocessor__num__imputer__strategy':['mean','median']}
#grid search
b2_test_clf = GridSearchCV(Baseline2, hyperparameters,cv = 5, scoring = "neg_root_mean_squared_error", n_jobs = -1, verbose = 2) 

In [ ]:
%%time
# Fit and tune model
b2_test_clf.fit(X_train, y_train)

In [ ]:
b2_test_clf.best_params_

In [ ]:
b2_val_clf = b2_test_clf.best_estimator_
b2_val_clf.fit(X_train2, y_train2)
b2_val_preds = b2_val_clf.predict(X_val)

# Evaluate the model
b2_val_mae = mean_absolute_error(y_val, b2_val_preds)
b2_val_mse = mean_squared_error(y_val, b2_val_preds)
b2_val_r2 = r2_score(y_val, b2_val_preds)

print('MAE:', b2_val_mae)
print("MSE: ",b2_val_mse)
print("R2: ",b2_val_r2)
print("NRMSE: ",-np.sqrt(b2_val_mse))

In [ ]:
b2_Te_clf = b2_test_clf.best_estimator_
b2_Te_clf.fit(X_train, y_train)
b2_test_preds = b2_Te_clf.predict(X_test)

# Evaluate the model
b2_test_mae = mean_absolute_error(y_test, b2_test_preds)
b2_test_mse = mean_squared_error(y_test, b2_test_preds)
b2_test_r2 = r2_score(y_test, b2_test_preds)

print('MAE:', b2_test_mae)
print("MSE: ",b2_test_mse)
print("R2: ",b2_test_r2)
print("NRMSE: ",-np.sqrt(b2_test_mse))

In [ ]:
x_tr = b2_test_clf.best_estimator_.named_steps["preprocessor"].fit_transform(X_train)#preprocess inputs
perm = PermutationImportance(b2_test_clf.best_estimator_.named_steps["model"]).fit(x_tr, y_train)#fit model
eli5.show_weights(perm, feature_names = X_train.columns.tolist())#show results

**insights:**
* carat is very important for this models prediction. 
* color and cut has no effect on the models predictions.
* only depth, z and clarity has a big effect on the model other featres less so.

### Baseline 2 Summary
The model was  cross validated 5 times for NRMSE, then validated and tested 1 time for all the metrics:
* **Mae**: validation set and test set around 800 and test performed slightly better. 
* **Mse**: validation set and test set around 1,400,000 and validation performed slightly better.
* **R2**: validation set and test set around 0.91 and validation performed slightly better.
* **NRMSE**: validation set and test set around 1,200 and validation performed slightly better.

The baseline is saved for comparison as a pandas dataframe: 

In [ ]:
baseline2 = pd.DataFrame({"val_mae": b2_val_mae,"val_mse": b2_val_mse,"val_r2": b2_val_r2,"val_nrmse": -np.sqrt(b2_val_mse),
                          "test_mae": b2_test_mae,"test_mse": b2_test_mse,"test_r2": b2_test_r2, "test_nrmse": -np.sqrt(b2_test_mse)}, index=["Baseline2"])
baseline2

The baselines are saved together for comparison as a pandas dataframe:

In [ ]:
Baselines = pd.concat([baseline1,baseline2])
Baselines